## Install necessary libraries

In [ ]:
pip install transformers shap datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 10.4 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2024.12.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is

## Import libraries

In [ ]:
import datasets
import numpy as np
import transformers
import shap

## Load dataset

In [ ]:
dataset = datasets.load_dataset("imdb", split="test")

# shorten the strings to fit into the pipeline model
short_data = [v[:500] for v in dataset["text"][:20]]

In [ ]:
print(dataset[0])

In [ ]:
print(short_data[0])

## Display the dataset

In [ ]:
import pandas as pd

# Create a pandas dataframe
df = pd.DataFrame(short_data)

# Display the head of the dataframe
print(df)

## Defining the NLP pipeline

In [ ]:
classifier = transformers.pipeline("sentiment-analysis", return_all_scores=True)

In [ ]:
# Example texts for sentiment analysis
texts = ["I love this movie! It was the best movie of all time", "I liked the first half but did not like the second half."]

# Perform sentiment analysis
results = classifier(texts)

print(results)

## Sentiment Analysis for first few data points

In [ ]:
print("1: ", short_data[0],"\n")
print("2: ", short_data[1],"\n")
print("3: ",short_data[2],"\n")
print("\n")
print(classifier(short_data[:3]))

## Find the model accuracy

In [ ]:
from sklearn.metrics import accuracy_score

# Predict the sentiment for the short data
predictions = classifier(short_data)

# Convert the predictions to labels
predicted_labels = []
for prediction in predictions:
    predicted_label = max(prediction, key=lambda x: x['score'])['label']
    predicted_labels.append(0 if predicted_label == 'NEGATIVE' else 1)  # Assuming 'NEGATIVE' is 0 and 'POSITIVE' is 1

true_labels = dataset["label"][:20]  # Get the true labels

# Calculate the accuracy
accuracy = accuracy_score(true_labels, predicted_labels)
print(f"Accuracy: {accuracy}")

print(true_labels)

print(predicted_labels)

Accuracy: 0.8
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [ ]:
print(dataset[12500])

{'text': "Previous reviewer Claudio Carvalho gave a much better recap of the film's plot details than I could. What I recall mostly is that it was just so beautiful, in every sense - emotionally, visually, editorially - just gorgeous.<br /><br />If you like movies that are wonderful to look at, and also have emotional content to which that beauty is relevant, I think you will be glad to have seen this extraordinary and unusual work of art.<br /><br />On a scale of 1 to 10, I'd give it about an 8.75. The only reason I shy away from 9 is that it is a mood piece. If you are in the mood for a really artistic, very romantic film, then it's a 10. I definitely think it's a must-see, but none of us can be in that mood all the time, so, overall, 8.75.", 'label': 1}


## Define SHAP explainer

In [ ]:
# define the explainer
explainer = shap.Explainer(classifier)

## Explain the predictions on first 2 samples

In [ ]:
# explain the predictions of the pipeline on the first two samples
shap_values = explainer(short_data[:2])

  0%|          | 0/498 [00:00<?, ?it/s]

PartitionExplainer explainer:  50%|█████     | 1/2 [00:00<?, ?it/s]

  0%|          | 0/498 [00:00<?, ?it/s]

PartitionExplainer explainer: 3it [04:00, 120.04s/it]


## SHAP force plot and text plot

In [ ]:
print(shap_values.shape)

(500, None, 2)


In [ ]:
shap.plots.text(shap_values[:, :, "POSITIVE"])